In this notebook,


**AssistantAgent's role:**
- Writes Python code for a given task
- Debugs the code if errors occur
- Suggests new code based on execution results


**UserProxyAgent's role:**
- Acts as a proxy for the human user and executes the code provided by AssistantAgent
- Generates automatic feedback based on the execution results


In [ ]:
from IPython.display import Image, display

from autogen.coding import LocalCommandLineCodeExecutor
from autogen import AssistantAgent, UserProxyAgent

In [ ]:
codellama_config_list = [
  {
    "model": "llama3:8b-instruct-q6_K",
    "base_url": "http://localhost:11434/v1",
    "api_key": "ollama",
  }
]


# create an AssistantAgent named "assistant"
assistant = AssistantAgent(
    name="assistant",
    llm_config={  
        "cache_seed": 42,  # seed for caching and reproducibility
        "config_list": codellama_config_list,  
        "temperature": 0,  # temperature for sampling
    }, 
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"), # LocalCommandLineCodeExecutor -> Enviroement: Shell, Platform: Local
    },
)

# the assistant receives a message from the user_proxy, which contains the task description
chat_res = user_proxy.initiate_chat(
    assistant,
    message="Create a plot showing the market caps of the top 7 publicly listed companies using data from Yahoo Finance.",
    summary_method="reflection_with_llm",
)

In [ ]:
print("Summary:", chat_res.summary)

In [ ]:
# followup question
user_proxy.send(
    recipient=assistant,
    message="""Plot a chart of the stock price change MTD. Save the data to stock_price_mtd.csv, and save the plot to stock_price_mtd.png.""",
)

In [ ]:
try:
    image = Image(filename="coding/stock_price_mtd.png")
    display(image)
except FileNotFoundError:
    print("Image not found. Please check the file name and modify if necessary.")

In [ ]:
# Path to your CSV file
file_path = "coding/stock_price_mtd.csv"
try:
    with open(file_path, mode="r", encoding="utf-8") as file:
        # Read each line in the file
        for line in file:
            # Split the line into a list using the comma as a separator
            row = line.strip().split(",")
            # Print the list representing the current row
            print(row)
except FileNotFoundError:
    print("File not found. Please check the file name and modify if necessary.")